# Fine-tuning T5 Japanese model on KDDI paired data for dialogue summarization

This script preprocesses data input from Splunk search and performs fine-tuning on T5 Japanese model

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
# import sys
# sys.path.insert(1, '/opt/conda/lib/python3.8/site-packages')

import json
import numpy as np
import pandas as pd
from pathlib import Path
import re
import math
import time
import copy
from tqdm import tqdm
import pandas as pd
import tarfile
# import neologdn
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer

# parameters for finetuning
MODEL_NAME = "sonoisa/t5-base-japanese"
# MODEL_NAME = "Huaibo/t5_dialog_jp"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

max_length_src = 400
max_length_target = 300

batch_size_train = 8
batch_size_valid = 8

epochs = 100
patience = 20

# path to save fine-tuned model
MODEL_DIRECTORY = "/srv/app/model/data/t5_finetune_jp"


In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

numpy version: 1.22.1
pandas version: 1.4.3


In [3]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [3]:
# from transformers import pipeline
# summarizer = pipeline("summarization", model="lidiya/bart-base-samsum")

## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a prepared dataset into this environment.

| makeresults
| eval text = "お客様：こんにちは、；従業者：こんにちは、；お客様、ご機嫌いかがでしょうか？；お客様：こんにちは、元気です。私はwifiサービスの性能に大きな問題を抱えています。先月から御社のwifiサービスを使い始めたのですが、購入したサービスのようにうまく機能していないのです。；従業者：今、ユーザー番号を確認します。お客様のユーザー番号は100145で、先月に作成されたものですが、よろしいですか？；お客様：はい ；従業者：お客様さん、何が問題なのか説明してもらえますか？；お客様：はい。私はあなたの無線LANのプレミアムバージョンに加入しており、それは通常の加入よりも高速であるはずです。しかし、とても遅いのです。；従業者：しかし、私たちのセンターから提供する無線LANサービスは、あなたが契約したパッケージの説明とまったく同じです。；お客様：私は自分の目で見た無線LANの速度を信じています。；従業者：申し訳ございませんが、お客様、私はあなたが嘘をついていると非難するつもりは全くありません、私はすぐにこの問題を解決します、あなたは私に詳細に説明することができますか？無線LANがあなたの期待に比べてどのように遅いですか？；お客様様：無線LANは時々オフラインで、物をダウンロードするのに時間がかかります。；従業者：ローカルネットワークに何らかの問題があるかもしれませんね。ルーターをチェックしていただけますか？；お客様: はい。 ；従業者： ごゆっくり、お客様。；お客様：ルーターの電源は入っていますが、ランプが点滅しています ；従業者：ランプが点滅するのは正常ではありません。ランプは接続の安定性を示すものですから、ランプが点滅するのは正常なことではありません。どのランプが点滅しているのか、またその点滅の速さを知る必要があるかもしれませんが、ご確認いただけますか？；お客様：はい ；従業者：アクティブという名前の光はありますか？；お客様：はい。それは点滅しているものです。；従業者：それはあなたのネットワークの問題であるべきですあなたはすべてのケーブルがうまく接続されていることを確認することができますか？；お客様：はい ；従業者：無線LANを再接続しようとしたことがありますか？；お客様：はい、私はやったし、私は問題があなたの側にあると思う　；従業者：今私はあなたのローカルネットワークのデータ入力を確認します"
| fit MLTKContainer algo=transformers_dialogsum_jp mode=stage epochs=100 text into app:transformers_dialogjp_model as text

In [4]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)    
    return df, param

In [13]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
df, param = stage("kddi_data")
print(df.head())
print(df.shape)
print(str(param))

                                                text  \
0  お客様：こんにちは、；従業者：こんにちは、；お客様、ご機嫌いかがでしょうか？；お客様：こんに...   
1  お客様：こんにちは、；従業者：こんにちは、；お客様、ご機嫌いかがでしょうか？；お客様：こんに...   
2                                                NaN   
3  お客様：こんにちは、；従業者：こんにちは、；お客様、ご機嫌いかがでしょうか？；お客様：こんに...   
4  お客様：こんにちは、；従業者：こんにちは、；お客様、ご機嫌いかがでしょうか？；お客様：こんに...   

                                             summary  
0  ▪対話経路: お客様 ▪対話者: 契約者本人 ▪RAS利用: なし ▪症状: WIFIが遅い...  
1  ▪対話経路: お客様 ▪対話者: 契約者本人 ▪RAS利用: なし ▪症状: WIFIが遅い...  
2  ▪対話経路: お客様 ▪対話者: 契約者本人 ▪RAS利用: なし ▪症状: WIFIが遅い...  
3  ▪対話経路: お客様 ▪対話者: 契約者本人 ▪RAS利用: なし ▪症状: WIFIが遅い...  
4                                                NaN  
(5, 2)
{'options': {'params': {'algo': 'transformers_dialogsum_jp', 'mode': 'stage', 'epochs': '100'}, 'args': ['text'], 'feature_variables': ['text'], 'model_name': 'transformers_dialogjp_model', 'output_name': 'text', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'handle_new_cat': 'default', 'max_distinct_cat_values': '100', 'ma

## Stage 2 - create and initialize a model

In [20]:
class T5FineTuner(nn.Module):
    
    def __init__(self):
        super().__init__()

        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

    def forward(
        self, input_ids, attention_mask=None, decoder_input_ids=None,
        decoder_attention_mask=None, labels=None
    ):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )
# initialize the model
# params: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    # Load English parser and text blob (for sentiment analysis)
    print("debug: start model initialization")
    model = T5FineTuner()
    model = model.to(device)
    print("debug: model initialized")
    return model

In [21]:
model = init(df,param)

debug: start model initialization
debug: model initialized


## Stage 3 - fit the model

We assume the input data frame has columns **text** and **summary**. The names can be changed later with rename(columns={'name': 'text'}
As preprocessing of summary data, we convert the bullet point-structured data into natural language. Special delimiters: u'\u25AA' and ":". Make sure to check if Japanese colon is used.

In [14]:
# Data preparation
def make_summary(doc):
    def new_dict():
        dictionary = {"対話経路":'',"対話者":'',"RAS利用":'',"症状":'',"発生頻度": '',"詳細":'',"本体SW":'',"通信環境":'',"周辺機器":'',"交換理由":'',"問診内容":''}
        return dictionary

    def parcing(doc):
        dictionary = new_dict()
        separator = u'\u25AA'
        infos = doc.split(separator)
        for info in infos:
            l = info.split(":")
            if len(l) == 2:
                dictionary[l[0]] = l[-1].rstrip().lstrip()
        return dictionary
    def formating(dictionary):
        line1 = dictionary["対話者"]+"である"+dictionary["対話経路"]+"から"+dictionary["症状"]+"という症状が"+dictionary["発生頻度"]+"で発生するとの連絡です。"
        line2 = dictionary["詳細"]+" "
        line3 = ""
        for key in ["RAS利用","本体SW","通信環境","周辺機器","交換理由","問診内容"]:
            if dictionary[key] != "":
                line3 += (key + "は" + dictionary[key] + "、")
        if line3 != "":
            line3 = line3[:-1] + "です"

        return line1 + line2 + line3
    return formating(parcing(doc))

def preprocess_text(text):
    text = re.sub(r'[\r\t\n\u3000]', '', text)
#     text = neologdn.normalize(text)
    text = text.lower()
    text = text.strip()
    return text


data = df.query('text.notnull()', engine='python').query('summary.notnull()', engine='python')

data = data.assign(
    text=lambda x: x.text.map(lambda y: preprocess_text(y)),
    summary=lambda x: x.summary.map(lambda y: make_summary(preprocess_text(y))))

In [15]:
data

,text,summary
0,お客様：こんにちは、；従業者：こんにちは、；お客様、ご機嫌いかがでしょうか？；お客様：こんに...,契約者本人であるお客様からwifiが遅いという症状が初めてで発生するとの連絡です。プレミアム...
1,お客様：こんにちは、；従業者：こんにちは、；お客様、ご機嫌いかがでしょうか？；お客様：こんに...,契約者本人であるお客様からwifiが遅いという症状が初めてで発生するとの連絡です。プレミアム...
3,お客様：こんにちは、；従業者：こんにちは、；お客様、ご機嫌いかがでしょうか？；お客様：こんに...,契約者本人であるお客様からwifiが遅いという症状が初めてで発生するとの連絡です。プレミアム...


In [17]:
# Data conversion
def convert_batch_data(train_data, valid_data, tokenizer):

    def generate_batch(data):

        batch_src, batch_tgt = [], []
        for src, tgt in data:
            batch_src.append(src)
            batch_tgt.append(tgt)

        batch_src = tokenizer(
            batch_src, max_length=max_length_src, truncation=True, padding="max_length", return_tensors="pt"
        )
        batch_tgt = tokenizer(
            batch_tgt, max_length=max_length_target, truncation=True, padding="max_length", return_tensors="pt"
        )

        return batch_src, batch_tgt

    train_iter = DataLoader(train_data, batch_size=batch_size_train, shuffle=True, collate_fn=generate_batch)
    valid_iter = DataLoader(valid_data, batch_size=batch_size_valid, shuffle=True, collate_fn=generate_batch)

    return train_iter, valid_iter

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME, is_fast=True)

print("Tokenizer initialized.")

X_train, X_test, y_train, y_test = train_test_split(
    data['text'], data['summary'], test_size=0.15, random_state=42, shuffle=True
)

train_data = [(src, tgt) for src, tgt in zip(X_train, y_train)]
valid_data = [(src, tgt) for src, tgt in zip(X_test, y_test)]

train_iter, valid_iter = convert_batch_data(train_data, valid_data, tokenizer)

print("data vectorization finished. Training data: " + str(len(train_data)) + ", Test data: " + str(len(valid_data)))

Tokenizer initialized.
data vectorization finished. Training data: 2, Test data: 1


In [24]:
# Training function
def train(model, data, optimizer, PAD_IDX):
    
    model.train()
    
    loop = 1
    losses = 0
    pbar = tqdm(data)
    for src, tgt in pbar:
                
        optimizer.zero_grad()
        
        labels = tgt['input_ids'].to(device)
        labels[labels[:, :] == PAD_IDX] = -100

        outputs = model(
            input_ids=src['input_ids'].to(device),
            attention_mask=src['attention_mask'].to(device),
            decoder_attention_mask=tgt['attention_mask'].to(device),
            labels=labels
        )
        loss = outputs['loss']

        loss.backward()
        optimizer.step()
        losses += loss.item()
        
        pbar.set_postfix(loss=losses / loop)
        loop += 1
        
    return losses / len(data)

# Loss function
def evaluate(model, data, PAD_IDX):
    
    model.eval()
    losses = 0
    with torch.no_grad():
        for src, tgt in data:

            labels = tgt['input_ids'].to(device)
            labels[labels[:, :] == PAD_IDX] = -100

            outputs = model(
                input_ids=src['input_ids'].to(device),
                attention_mask=src['attention_mask'].to(device),
                decoder_attention_mask=tgt['attention_mask'].to(device),
                labels=labels
            )
            loss = outputs['loss']
            losses += loss.item()
        
    return losses / len(data)


In [27]:
# returns a fit info json object
def fit(model,df,param):
    optimizer = optim.Adam(model.parameters())

    PAD_IDX = tokenizer.pad_token_id
    best_loss = float('Inf')
    best_model = None
    counter = 1

    print("Start training.")

    for loop in range(1, epochs + 1):

        start_time = time.time()

        loss_train = train(model=model, data=train_iter, optimizer=optimizer, PAD_IDX=PAD_IDX)

        elapsed_time = time.time() - start_time

        loss_valid = evaluate(model=model, data=valid_iter, PAD_IDX=PAD_IDX)

        print('[{}/{}] train loss: {:.4f}, valid loss: {:.4f} [{}{:.0f}s] counter: {} {}'.format(
            loop, epochs, loss_train, loss_valid,
            str(int(math.floor(elapsed_time / 60))) + 'm' if math.floor(elapsed_time / 60) > 0 else '',
            elapsed_time % 60,
            counter,
            '**' if best_loss > loss_valid else ''
        ))

        if best_loss > loss_valid:
            best_loss = loss_valid
            best_model = copy.deepcopy(model)
            counter = 1
        else:
            if counter > patience:
                break

            counter += 1

    print("finished training.")
    returns = {}
    return returns

## Stage 4 - apply the model

This stage should be called after Stage 5 (saving model) and 6 (loading model)

In [21]:
def apply(model,df,param):
    
    def generate_text_from_model(text, trained_model, tokenizer, num_return_sequences=1):

        trained_model.eval()

        text = preprocess_text(text)
        batch = tokenizer(
            [text], max_length=max_length_src, truncation=True, padding="longest", return_tensors="pt"
        )

        outputs = trained_model.generate(
            input_ids=batch['input_ids'].to(device),
            attention_mask=batch['attention_mask'].to(device),
            max_length=max_length_target,
            repetition_penalty=8.0,  
            num_beams=10,
            num_return_sequences=num_return_sequences,  
        )

        generated_texts = [
            tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False) for ids in outputs
        ]

        return generated_texts

    index = 100
    body = valid_data[index][0]
    summaries = valid_data[index][1]
    generated_texts = generate_text_from_model(
        text=body, trained_model=model["summarizer"], tokenizer=model["tokenizer"], num_return_sequences=1
    )
    print('Summarization')
    print('\n'.join(generated_texts[0].split('。')))
    print()
    print('ground truth')
    print('\n'.join(summaries.split('。')))
    print()
    print('original text')
    print(body)
        
    return generated_texts[0]

In [22]:
returns = apply(model,df,param)

debug: start apply
debug: read input
numpy version: 1.22.1
debug: start tokenizing
debug: finish tok and start summarizing
debug: finish summarizing and start decoding
debug: finish decoding
debug: finished
無線のプレミアムバージョンに加入しているユーザー番号は100145。通常加入よりも高速であるはずなのに、とても遅いという。お客様が契約したパッケージの説明とまったく同じだそう


## Stage 5 - save the model

In [14]:
# save model to name in expected convention "<algo_name>_<model_name>.h5"
def save(path):
    model_dir_path = Path(path)
    tokenizer.save_pretrained(model_dir_path)
    print("tokenizer saved.")
    best_model.model.save_pretrained(model_dir_path)
    print("model saved. Successfully finished.")

In [ ]:
save(MODEL_DIRECTORY)

## Stage 6 - load the model

In [15]:
# load model from name in expected convention "<algo_name>_<model_name>.h5"
def load(path):
    model = {}
    model["tokenizer"] = T5Tokenizer.from_pretrained(path)
    model["summarizer"] = T5ForConditionalGeneration.from_pretrained(path)
    return model

In [ ]:
model = load(MODEL_DIRECTORY)

## Stage 7 - provide a summary of the model

In [16]:
# return model summary
def summary(model=None):
    returns = {}
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code